In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import warnings

warnings.simplefilter('ignore')

# RID OSPAR Reporting Template

**Note:** Despite the [changes to the programme](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/programme_changes_2017-18.ipynb) for 2017 to 2020, we have decided to continue to submit data to OSPAR using the old divisions (see e-mail from Øyvind received 15.10.2018 at 15.51). The code in this notebook therefore uses the column named `'old_rid_code'` to allocate fluxes to the correct regions.

The final step in the annual RID reporting is to complete and submit an overall summary template to OSPAR. A blank template and guideline documentation were provided by Eva (see e-mail received 04/09/2017 at 13.07) and the 2015 template created by Tore is here:

K:\Avdeling\Vass\316_Miljøinformatikk\Prosjekter\RID\2016\OSPAR\Norway_2015.xlsx

**Note:** Tore previously calculated "upper" load estimates by setting LOD values equal to the LOD itself, and "lower" estimates by setting LOD values equal to zero. This year we have handled LOD values more explicity, by calculating a single estimate as described [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/rid_data_exploration.ipynb#2.3.-Calculate-loads). We therefore only need to fill-in entires for the "mean" rows.

Most of the information required for the template has already been reported in other tables, especially the following:

 * *Table_3_2016.docx* <br><br>
 
 * *loads_and_flows_all_sites_2016.csv* <br><br>
 
 * *concs_and_flows_rid_11-36_2016.csv*

However, some information is not available from these tables, and in previous years Tore has simply left these cells blank. The code here does the same, although in principle we could fill-in the whole thing with a bit more work.

## 1. Get summary data

The notebook [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/summary_table_2016.ipynb) calculates summary values for Table 3. Rather than reading these values from Word, it is easier to repeat the code to summarise the raw data again. This is the data that needs writing to the OSPAR template.

### 1.1. Monitored areas

In [2]:
# Set year of interest
year = 2018

In [3]:
# Read data
in_csv = r'../../../Results/Loads_CSVs/loads_and_flows_all_sites_%s.csv' % year
mon_df = pd.read_csv(in_csv)
del mon_df['new_rid_group']

# Group by OSPAR region
mon_df1 = mon_df.groupby(['ospar_region', 'old_rid_group']).sum()

# Totals for Norway
mon_df2 = mon_df.groupby('old_rid_group').sum().reset_index()
mon_df2['ospar_region'] = 'NORWAY'
mon_df2.set_index(['ospar_region', 'old_rid_group'], inplace=True)

# Combine
mon_df = pd.concat([mon_df1, mon_df2], axis=0)

# Cols of interest
cols = [i for i in mon_df.columns if i.split('_')[1] != 'Est']
mon_df = mon_df[cols]
del mon_df['station_id'], mon_df['mean_q_1000m3/day']

# Convert units
mon_df['Hg_kg'] = mon_df['Hg_kg']/1000. # kg to tonnes
mon_df['NH4-N_tonnes'] = mon_df['NH4-N_tonnes']/1000. # tonnes to ktonnes
mon_df['NO3-N_tonnes'] = mon_df['NO3-N_tonnes']/1000. # tonnes to ktonnes
mon_df['TOTN_tonnes'] = mon_df['TOTN_tonnes']/1000. # tonnes to ktonnes
mon_df['TOTP_tonnes'] = mon_df['TOTP_tonnes']/1000. # tonnes to ktonnes
mon_df['PO4-P_tonnes'] = mon_df['PO4-P_tonnes']/1000. # tonnes to ktonnes
mon_df['SPM_tonnes'] = mon_df['SPM_tonnes']/1000. # tonnes to ktonnes

# Units are correct, so remove
mon_df.columns = [i.split('_')[0] for i in mon_df.columns]

mon_df.round(0)

Ag    As   Cd    Cr     Cu   Hg  NH4-N  \
ospar_region        old_rid_group                                            
LOFOTEN-BARENTS SEA rid_108        0.0   1.0  0.0   1.0    6.0  0.0    0.0   
                    rid_11         0.0   0.0  0.0   1.0    1.0  0.0    0.0   
                    rid_36         0.0   3.0  0.0   4.0  117.0  0.0    0.0   
NORTH SEA           rid_108        0.0   2.0  0.0   1.0   12.0  0.0    0.0   
                    rid_11         0.0   0.0  0.0   0.0    2.0  0.0    0.0   
                    rid_36         0.0   2.0  0.0   3.0   11.0  0.0    0.0   
NORWEGIAN SEA2      rid_108        0.0   1.0  0.0   1.0    6.0  0.0    0.0   
                    rid_11         0.0   1.0  0.0   1.0   12.0  0.0    0.0   
                    rid_36         0.0   3.0  0.0   8.0   25.0  0.0    0.0   
SKAGERAK            rid_108        0.0   1.0  0.0   1.0    3.0  0.0    0.0   
                    rid_11         0.0   8.0  1.0   9.0   50.0  0.0    1.0   
                    rid_36         0.0   2.0  0.0   1.0    6.0  0.0    0.0   
NORWAY              rid_108        0.0   4.0  0.0   4.0   26.0  0.0    0.0   
                    rid_11         0.0   9.0  1.0  10.0   65.0  0.0    1.0   
                    rid_36         0.0  10.0  1.0  16.0  160.0  0.0    1.0   

                                   NO3-N     Ni  PO4-P    Pb    SPM      SiO2  \
ospar_region        old_rid_group                                               
LOFOTEN-BARENTS SEA rid_108          0.0    5.0    0.0   0.0   10.0   24334.0   
                    rid_11           0.0    1.0    0.0   0.0   11.0   12663.0   
                    rid_36           1.0  113.0    0.0   2.0   28.0   82178.0   
NORTH SEA           rid_108          3.0    8.0    0.0   3.0   28.0   31505.0   
                    rid_11           1.0    1.0    0.0   0.0    5.0    3697.0   
                    rid_36           4.0    4.0    0.0   4.0   48.0   37606.0   
NORWEGIAN SEA2      rid_108          1.0    4.0    0.0   1.0   24.0   20003.0   
                    rid_11           1.0    3.0    0.0   0.0   13.0   12505.0   
                    rid_36           2.0   18.0    0.0   2.0  100.0   65771.0   
SKAGERAK            rid_108          1.0    2.0    0.0   1.0   11.0    9219.0   
                    rid_11          15.0   27.0    0.0   9.0  215.0  149378.0   
                    rid_36           1.0    3.0    0.0   3.0   15.0   19525.0   
NORWAY              rid_108          6.0   19.0    0.0   6.0   73.0   85061.0   
                    rid_11          16.0   33.0    0.0  10.0  245.0  178242.0   
                    rid_36           8.0  139.0    0.0  11.0  191.0  205079.0   

                                        TOC  TOTN  TOTP     Zn  
ospar_region        old_rid_group                               
LOFOTEN-BARENTS SEA rid_108         26121.0   1.0   0.0    7.0  
                    rid_11           8767.0   0.0   0.0    0.0  
                    rid_36          45479.0   3.0   0.0   33.0  
NORTH SEA           rid_108         31443.0   6.0   0.0   33.0  
                    rid_11           4831.0   1.0   0.0    4.0  
                    rid_36          46322.0   7.0   0.0   55.0  
NORWEGIAN SEA2      rid_108         20876.0   2.0   0.0    9.0  
                    rid_11          14849.0   1.0   0.0   25.0  
                    rid_36          67315.0   7.0   0.0   61.0  
SKAGERAK            rid_108         14964.0   2.0   0.0   21.0  
                    rid_11         170042.0  23.0   0.0  164.0  
                    rid_36          46738.0   3.0   0.0   36.0  
NORWAY              rid_108         93404.0  11.0   0.0   70.0  
                    rid_11         198490.0  25.0   1.0  194.0  
                    rid_36         205854.0  20.0   0.0  184.0

### 1.2. Unmonitored areas

In [4]:
# Read data
in_csv = r'../../../Results/Unmon_loads/unmon_loads_%s.csv' % year
umon_df = pd.read_csv(in_csv, index_col=0)

# Rename cols
umon_df.columns = [i.replace('RENSEANLEGG', 'sew') for i in umon_df.columns]
umon_df.columns = [i.replace('INDUSTRI', 'ind') for i in umon_df.columns]
umon_df.columns = [i.replace('_tonn', '') for i in umon_df.columns]
umon_df.columns = [i.replace('AQUAKULTUR', 'fish') for i in umon_df.columns]

# Convert Hg to kgs
umon_df['sew_Hg'] = umon_df['sew_Hg']*1000
umon_df['ind_Hg'] = umon_df['ind_Hg']*1000

umon_df.round(0)

,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p,sew_po4,...,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg,fish_Cu
ospar_region,,,,,,,,,,,,,,,,,,,,,
NORWAY,448260.0,13095.0,1099.0,2582.0,262.0,60256.0,10404.0,36655.0,731.0,659.0,...,12908.0,0.0,0.0,0.0,4.0,13.0,2.0,1.0,8.0,1217.0
LOFOTEN-BARENTS SEA,127960.0,1577.0,176.0,128.0,6.0,14879.0,2574.0,5930.0,116.0,106.0,...,2691.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,301.0
NORTH SEA,163155.0,3551.0,476.0,487.0,95.0,18894.0,3249.0,15224.0,246.0,286.0,...,3739.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,1.0,380.0
NORWEGIAN SEA2,146838.0,2687.0,350.0,992.0,113.0,26442.0,4574.0,12480.0,281.0,210.0,...,4584.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,6.0,536.0
SKAGERAK,10308.0,5280.0,97.0,975.0,48.0,40.0,7.0,3021.0,88.0,58.0,...,1894.0,0.0,0.0,0.0,2.0,8.0,1.0,0.0,1.0,1.0


### 1.3. Loads for 11 main rivers

In [5]:
# Read data
in_csv = r'../../../Results/Loads_CSVs/loads_and_flows_all_sites_%s.csv' % year
rid11_df = pd.read_csv(in_csv, index_col=0)
del rid11_df['new_rid_group']

# Get data for RID11
rid11_df = rid11_df.query('old_rid_group == "rid_11"')

# Tidy
del rid11_df['station_code'], rid11_df['station_name']
del rid11_df['old_rid_group'], rid11_df['ospar_region']
del rid11_df['mean_q_1000m3/day']

cols = [i for i in rid11_df.columns if i.split('_')[1] != 'Est']
rid11_df = rid11_df[cols]

# Convert units
rid11_df['Hg_kg'] = rid11_df['Hg_kg']/1000. # kg to tonnes
rid11_df['NH4-N_tonnes'] = rid11_df['NH4-N_tonnes']/1000. # tonnes to ktonnes
rid11_df['NO3-N_tonnes'] = rid11_df['NO3-N_tonnes']/1000. # tonnes to ktonnes
rid11_df['TOTN_tonnes'] = rid11_df['TOTN_tonnes']/1000. # tonnes to ktonnes
rid11_df['TOTP_tonnes'] = rid11_df['TOTP_tonnes']/1000. # tonnes to ktonnes
rid11_df['PO4-P_tonnes'] = rid11_df['PO4-P_tonnes']/1000. # tonnes to ktonnes
rid11_df['SPM_tonnes'] = rid11_df['SPM_tonnes']/1000. # tonnes to ktonnes

# Tidy cols
rid11_df.columns = [i.split('_')[0] for i in rid11_df.columns]

rid11_df.head()

,Ag,As,Cd,Cr,Cu,Hg,NH4-N,NO3-N,Ni,PO4-P,Pb,SPM,SiO2,TOC,TOTN,TOTP,Zn
station_id,,,,,,,,,,,,,,,,,
29612,0.024423,1.550438,0.075695,1.096639,6.560358,0.000000,0.098972,2.546332,4.284334,0.021437,1.180816,21.621902,29215.268094,32330.351209,4.062213,0.055955,20.254384
29779,0.000000,0.252603,0.000000,0.505210,1.352785,0.001137,0.007133,0.117960,0.699360,0.009227,0.029662,11.260444,12663.196285,8767.417028,0.479765,0.020126,0.459972
29821,0.000000,0.271211,0.021575,0.184911,1.255239,0.001031,0.023009,0.378198,1.033374,0.004675,0.216557,2.428687,3357.522938,3854.602467,0.600962,0.009759,3.312905
29782,0.000000,0.606220,0.015739,0.622476,1.851907,0.000000,0.005707,0.244700,1.796032,0.003659,0.366180,11.354839,7085.741882,8509.870908,0.591907,0.017204,2.367360
36225,0.000512,0.011721,0.001137,0.015231,0.100386,0.000027,0.000956,0.032278,0.029580,0.002011,0.016361,0.762104,240.197737,184.023884,0.046228,0.002395,0.365054


## 2. Fill-in template

A copy of the 2018 template from Eva is here:

    ./Results/OSPAR_Template/01_OSPAR_Norway_2018.xlsx

In [6]:
# Copy of template to update
temp_path = r'../../../Results/OSPAR_Template/01_OSPAR_Norway_%s.xlsx' % year

In [7]:
def update_spreadsheet_point_sources(xlsx, sheet, pars, src, df):
    """ Update the OSPAR template for point source data.
    
    Args:
        xslx:  Str. Path to Excel template
        sheet: Str. Sheet name to update
        pars:  List. Parameter names in template to fill-in
        src:   Str. Type of input ('sew', 'ind', 'fish')
        df:    Dataframe. Values to fill-in
        
    Returns:
        None. The template is updated and saved.
    """
    import pandas as pd
    from openpyxl import load_workbook
    
    # Map Excel headings to df cols
    par_dict = {'SPM':'S.P.M.',
                'TOC':'TOC',
                'PO4-P':'po4',
                'P-Total':'p',
                'NO3-N':'no3', 
                'NH4-N':'nh4',
                'N-Total':'n',
                'As':'As',
                'Pb':'Pb',
                'Cd':'Cd', 
                'Cu':'Cu',
                'Zn':'Zn',
                'Ni':'Ni',
                'Total Cr':'Cr',
                'Hg':'Hg'}

    # Map template names to df names
    names_dict = {'Norwegian Sea (NO)':'NORWEGIAN SEA2',
                  'Barents Sea (NO)':'LOFOTEN-BARENTS SEA',
                  'Skagerrak (NO)':'SKAGERAK',
                  'North Sea (NO)':'NORTH SEA',
                  'Norway Total':'NORWAY'}

    # Open new file and get sheet
    wb = load_workbook(filename=xlsx)
    ws = wb[sheet]

    # Get row numbers
    row_dict = {}
    for item in ws['B12':'B%s' % ws.max_row]:
        # Get cell properties
        cell = item[0]
        name = cell.value
        row = cell.row
        row_dict[name] = row

    # Get col numbers
    col_dict = {}
    for cell in ws['E9':'AK9'][0]:
        # Get cell properties
        par = cell.value
        col = cell.column
        col_dict[par] = col

    # Update spreadsheet
    for reg in names_dict.keys():
        for par in pars:
            # Get value from df
            val = df.ix[names_dict[reg], '%s_%s' % (src, par_dict[par])]

            # Get cell co-ords
            row = row_dict[reg] + 2
            col = col_dict[par]
           
            # Write value
            ws.cell(row=row, column=col).value = val

    # Save
    wb.save(xlsx)

### 2.1. Sheet 5a: Sewage effluents

Note these are the values from Table 3 for **unmonitored areas** and not the total sewage inputs for each of the OSPAR areas. This is the same as what Tore reported previously.

In [8]:
# Get Sewage data
cols = [i for i in umon_df.columns if i.split('_')[0]=='sew']
sew_df = umon_df[cols].copy()

# Convert units
sew_df['sew_Hg'] = sew_df['sew_Hg']/1000. # kg to tonnes
sew_df['sew_nh4'] = sew_df['sew_nh4']/1000. # tonnes to ktonnes
sew_df['sew_no3'] = sew_df['sew_no3']/1000. # tonnes to ktonnes
sew_df['sew_n'] = sew_df['sew_n']/1000. # tonnes to ktonnes
sew_df['sew_po4'] = sew_df['sew_po4']/1000. # tonnes to ktonnes
sew_df['sew_p'] = sew_df['sew_p']/1000. # tonnes to ktonnes
sew_df['sew_S.P.M.'] = sew_df['sew_S.P.M.']/1000. # tonnes to ktonnes

sew_df

,sew_n,sew_p,sew_po4,sew_no3,sew_nh4,sew_S.P.M.,sew_As,sew_Pb,sew_Cd,sew_Cu,sew_Zn,sew_Ni,sew_Cr,sew_Hg
ospar_region,,,,,,,,,,,,,,
NORWAY,13.094681,1.098523,0.659114,0.654734,9.821010,12.907770,0.305828,0.243865,0.015623,4.010682,13.174976,1.654495,0.58213,0.008478
LOFOTEN-BARENTS SEA,1.576575,0.175979,0.105587,0.078829,1.182432,2.690578,0.005658,0.004878,0.000308,0.153628,0.428932,0.040349,0.01089,0.000219
NORTH SEA,3.550676,0.476355,0.285813,0.177534,2.663007,3.738730,0.105146,0.064361,0.003397,0.716973,2.959769,0.297517,0.05688,0.000837
NORWEGIAN SEA2,2.687039,0.349592,0.209755,0.134352,2.015279,4.583968,0.040466,0.046799,0.002013,0.920609,1.747913,0.121765,0.28380,0.006283
SKAGERAK,5.280390,0.096597,0.057958,0.264019,3.960292,1.894494,0.154558,0.127827,0.009905,2.219472,8.038362,1.194864,0.23056,0.001139


In [9]:
# Update sheet 5a
pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu']

update_spreadsheet_point_sources(temp_path, '5a', pars, 'sew', sew_df)

### 2.2. Sheet 5b: Industrial effluents

In [10]:
# Get industrial data
cols = [i for i in umon_df.columns if i.split('_')[0]=='ind']
ind_df = umon_df[cols].copy()

# Convert units
ind_df['ind_Hg'] = ind_df['ind_Hg']/1000. # kg to tonnes
ind_df['ind_nh4'] = ind_df['ind_nh4']/1000. # tonnes to ktonnes
ind_df['ind_no3'] = ind_df['ind_no3']/1000. # tonnes to ktonnes
ind_df['ind_n'] = ind_df['ind_n']/1000. # tonnes to ktonnes
ind_df['ind_po4'] = ind_df['ind_po4']/1000. # tonnes to ktonnes
ind_df['ind_p'] = ind_df['ind_p']/1000. # tonnes to ktonnes
ind_df['ind_S.P.M.'] = ind_df['ind_S.P.M.']/1000. # tonnes to ktonnes

ind_df

,ind_n,ind_p,ind_po4,ind_no3,ind_nh4,ind_S.P.M.,ind_TOC,ind_As,ind_Pb,ind_Cd,ind_Cu,ind_Zn,ind_Ni,ind_Cr,ind_Hg
ospar_region,,,,,,,,,,,,,,,
NORWAY,2.581831,0.262263,0.157358,0.129092,1.936373,17.467110,974.3877,1.655479,1.060397,0.076911,5.292689,16.919169,5.071773,1.791278,0.012042
LOFOTEN-BARENTS SEA,0.127990,0.006160,0.003696,0.006400,0.095993,4.318558,205.8439,0.000085,0.000563,0.000009,0.000084,0.000547,0.000570,0.001164,0.000045
NORTH SEA,0.486926,0.094579,0.056747,0.024346,0.365194,8.226270,595.2192,1.406532,0.741851,0.044532,0.577438,4.478515,3.080723,1.021131,0.005158
NORWEGIAN SEA2,0.992103,0.113073,0.067843,0.049605,0.744078,3.775973,83.9946,0.012456,0.036463,0.011255,0.508623,3.692721,0.137741,0.053962,0.000549
SKAGERAK,0.974812,0.048451,0.029071,0.048741,0.731109,1.146308,89.3300,0.236406,0.281520,0.021115,4.206543,8.747386,1.852738,0.715021,0.006289


In [11]:
# Update sheet 5b
pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu', 'TOC']

update_spreadsheet_point_sources(temp_path, '5b', pars, 'ind', ind_df)

### 2.3. Sheet 5c: Aquaculture discharges

In [12]:
# Get fish data
cols = [i for i in umon_df.columns if i.split('_')[0]=='fish']
fish_df = umon_df[cols].copy()

# Convert units
fish_df['fish_nh4'] = fish_df['fish_nh4']/1000. # tonnes to ktonnes
fish_df['fish_no3'] = fish_df['fish_no3']/1000. # tonnes to ktonnes
fish_df['fish_n'] = fish_df['fish_n']/1000. # tonnes to ktonnes
fish_df['fish_po4'] = fish_df['fish_po4']/1000. # tonnes to ktonnes
fish_df['fish_p'] = fish_df['fish_p']/1000. # tonnes to ktonnes

fish_df

,fish_n,fish_p,fish_po4,fish_no3,fish_nh4,fish_Cu
ospar_region,,,,,,
NORWAY,60.255949,10.404244,7.178929,6.628154,48.204759,1217.000000
LOFOTEN-BARENTS SEA,14.878861,2.574264,1.776242,1.636675,11.903089,300.837800
NORTH SEA,18.894334,3.249395,2.242082,2.078377,15.115467,379.735951
NORWEGIAN SEA2,26.442431,4.573663,3.155827,2.908667,21.153945,535.617270
SKAGERAK,0.040324,0.006922,0.004776,0.004436,0.032259,0.808979


In [13]:
# Update sheet 5c
pars = ['NH4-N', 'NO3-N', 'P-Total', 'PO4-P', 'N-Total', 'Cu']

update_spreadsheet_point_sources(temp_path, '5c', pars, 'fish', fish_df)

### 2.4. Sheet 5d: Other discharges

This sheet is left blank

### 2.5. Sheet 5e: Total direct discharges

The sum of sewage, industrial and fish-farm discharges.

In [14]:
# Combine sew, ind and fish, then aggregate
for df in [sew_df, ind_df, fish_df]:
    df.reset_index(inplace=True)
    df.columns = [i.split('_')[1] for i in df.columns]

td_df = pd.concat([sew_df, ind_df, fish_df], axis=0, sort=True)
td_df = td_df.groupby('region').sum()

td_df.columns = ['td_'+i for i in td_df.columns]

td_df

,td_As,td_Cd,td_Cr,td_Cu,td_Hg,td_Ni,td_Pb,td_S.P.M.,td_TOC,td_Zn,td_n,td_nh4,td_no3,td_p,td_po4
region,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,0.005743,0.000317,0.012054,300.991512,0.000264,0.040919,0.005441,7.009136,205.8439,0.429479,16.583426,13.181513,1.721903,2.756403,1.885526
NORTH SEA,1.511678,0.047929,1.078011,381.030362,0.005995,3.378240,0.806212,11.965000,595.2192,7.438284,22.931936,18.143669,2.280257,3.820329,2.584643
NORWAY,1.961307,0.092534,2.373408,1226.303371,0.020520,6.726268,1.304262,30.374880,974.3877,30.094145,75.932461,59.962143,7.411980,11.765030,7.995400
NORWEGIAN SEA2,0.052922,0.013268,0.337762,537.046503,0.006832,0.259506,0.083262,8.359941,83.9946,5.440634,30.121574,23.913302,3.092625,5.036327,3.433426
SKAGERAK,0.390964,0.031020,0.945581,7.234994,0.007428,3.047602,0.409347,3.040802,89.3300,16.785748,6.295526,4.723660,0.317196,0.151971,0.091805


In [15]:
# Update sheet 5e
pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu', 'TOC']

update_spreadsheet_point_sources(temp_path, '5e', pars, 'td', td_df)

### 2.6. Sheet 6a: Monitored rivers

**Does "Inner Oslofjord" in the template correspond to "Alna"?** If so, I can fill-in one additional row in this table.

In [16]:
mon_df.reset_index(inplace=True)
tot_df = mon_df.groupby('ospar_region').sum()
trib_df = mon_df[mon_df['old_rid_group']!='rid_11'].groupby('ospar_region').sum()

In [17]:
def update_spreadsheet_monitored_rivers(xlsx, sheet, pars, df_dict):
    """ Update the OSPAR template for monitored rivers.
    
    Args:
        xslx:    Str. Path to Excel template
        sheet:   Str. Sheet name to update
        pars:    List. Parameter names in template to fill-in
        df_dict: Dict. {'tot':tot_df, 'trib':trib_df, 'main':rid11_df}
                 Values to fill-in
        
    Returns:
        None. The template is updated and saved.
    """
    import pandas as pd
    from openpyxl import load_workbook

    # Map Excel headings to df cols
    par_dict = {'SPM':'SPM',
                'TOC':'TOC',
                'PO4-P':'PO4-P',
                'P-Total':'TOTP',
                'NO3-N':'NO3-N', 
                'NH4-N':'NH4-N',
                'N-Total':'TOTN',
                'As':'As',
                'Pb':'Pb',
                'Cd':'Cd', 
                'Cu':'Cu',
                'Zn':'Zn',
                'Ni':'Ni',
                'Total Cr':'Cr',
                'Hg':'Hg'}
  
    # Map template names to df names and rows
    names_dict = {'Norwegian Sea (NO)':('NORWEGIAN SEA2', 'tot'),
                  'Barents Sea (NO)':('LOFOTEN-BARENTS SEA', 'tot'),
                  'Skagerrak (NO)':('SKAGERAK', 'tot'),
                  'North Sea (NO)':('NORTH SEA', 'tot'),
                  'Norway Total':('NORWAY', 'tot'),
                  'Tributary Rivers - Norwegian Sea':('NORWEGIAN SEA2', 'trib'),
                  'Tributary Rivers - Barents Sea':('LOFOTEN-BARENTS SEA', 'trib'),
                  'Tributary Rivers - Skagerak':('SKAGERAK', 'trib'),
                  'Tributary Rivers - North Sea':('NORTH SEA', 'trib'),
                  'Orkla':(29778, 'main'),
                  'Vefsna':(29782, 'main'),
                  'Alta':(29779, 'main'),
                  'Glomma':(29617, 'main'),
                  'Drammenselva':(29612, 'main'),
                  u'Numedalslågen':(29615, 'main'),
                  'Skienselva':(29613, 'main'),
                  'Otra':(29614, 'main'),
                  'Orreelva':(29783, 'main'),
                  'Vosso':(29821, 'main')}

    # Open new file and get sheet
    wb = load_workbook(filename=xlsx)
    ws = wb[sheet]

    # Get row numbers
    row_dict = {}
    for item in ws['B12':'B%s' % ws.max_row]:
        # Get cell properties
        cell = item[0]
        name = cell.value
        row = cell.row
        row_dict[name] = row

    # Get col numbers
    col_dict = {}
    for cell in ws['E9':'AK9'][0]:
        # Get cell properties
        par = cell.value
        col = cell.column
        col_dict[par] = col

    # Update spreadsheet
    for reg in names_dict.keys():
        df_idx, df_name = names_dict[reg]
        
        # Get df 
        df = df_dict[df_name]
        
        for par in pars:
            # Get value from df
            val = df.ix[df_idx, par_dict[par]]

            # Get cell co-ords
            row = row_dict[reg] + 2
            col = col_dict[par]

            # Write value
            ws.cell(row=row, column=col).value = val

    # Save
    wb.save(xlsx)

In [18]:
# Update sheet 6a
pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu', 'TOC']
df_dict = {'tot':tot_df,
           'main':rid11_df,
           'trib':trib_df}

update_spreadsheet_monitored_rivers(temp_path, '6a', pars, df_dict)

### 2.7. Sheet 6b: Unmonitored areas

In [19]:
# Get diff data
cols = [i for i in umon_df.columns if i.split('_')[0]=='diff']
diff_df = umon_df[cols].copy()

# Convert units
diff_df['diff_nh4'] = diff_df['diff_nh4']/1000. # tonnes to ktonnes
diff_df['diff_no3'] = diff_df['diff_no3']/1000. # tonnes to ktonnes
diff_df['diff_n'] = diff_df['diff_n']/1000. # tonnes to ktonnes
diff_df['diff_po4'] = diff_df['diff_po4']/1000. # tonnes to ktonnes
diff_df['diff_p'] = diff_df['diff_p']/1000. # tonnes to ktonnes

diff_df

,diff_n,diff_p,diff_po4,diff_no3,diff_nh4
ospar_region,,,,,
NORWAY,36.654556,0.731307,0.179901,22.909098,2.016001
LOFOTEN-BARENTS SEA,5.930259,0.115806,0.028488,3.706412,0.326164
NORTH SEA,15.223662,0.246341,0.060600,9.514789,0.837301
NORWEGIAN SEA2,12.480046,0.281042,0.069136,7.800029,0.686403
SKAGERAK,3.020589,0.088118,0.021677,1.887868,0.166132


In [20]:
# Update sheet 6b
pars = ['NH4-N', 'NO3-N', 'P-Total', 'PO4-P', 'N-Total']

update_spreadsheet_point_sources(temp_path, '6b', pars, 'diff', diff_df)

### 2.8. Sheet 6c: Total inputs

**Note:** See e-mail from Csilla received 07/11/2017 at 13.43. This table should **not** include "point" discharges (`td_df`) - it's just the sum of tables 6a and 6b.

In [21]:
# Rename cols in diff_df
col_map = {'diff_n':'TOTN',
           'diff_p':'TOTP',
           'diff_po4':'PO4-P',
           'diff_no3':'NO3-N',
           'diff_nh4':'NH4-N'}
diff_df.columns = [col_map[i] for i in diff_df.columns]

# Add to total_df
for col in diff_df.columns:
    tot_df[col] = tot_df[col] + diff_df[col]

# Update sheet 6c
pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu', 'TOC']

df_dict = {'tot':tot_df,
           'main':rid11_df,
           'trib':trib_df}

update_spreadsheet_monitored_rivers(temp_path, '6c', pars, df_dict)

The following two cells are not longer used as I originally misunderstood what table 6c represents.

In [ ]:
## Standardise col names
#diff_df.columns = [i.split('_')[1] for i in diff_df.columns]
#td_df.columns = [i.split('_')[1] for i in td_df.columns]
#td_df.index.name = 'ospar_region'
#
#col_dict = {'SPM':'S.P.M.',
#            'TOTN':'n',
#            'NH4-N':'nh4',
#            'NO3-N':'no3',
#            'TOTP':'p',
#            'PO4-P':'po4'}
#for col in col_dict.keys():
#    new_col = col_dict[col]
#    tot_df[new_col] = tot_df[col]
#    del tot_df[col]
#    
## Reset index
#diff_df.reset_index(inplace=True)
#tot_df.reset_index(inplace=True)
#td_df.reset_index(inplace=True)
#
## Concat and aggregate
##tot_df = pd.concat([diff_df, td_df, tot_df], axis=0).groupby('ospar_region').sum() # See comment above
#tot_df = pd.concat([diff_df, tot_df], axis=0).groupby('ospar_region').sum()
#
## Rename cols
#tot_df.columns = ['tot_'+i for i in tot_df.columns]
#
#tot_df

In [ ]:
## Update sheet 6c
#pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
#        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
#        'N-Total', 'Hg', 'Cu', 'TOC']
#
#update_spreadsheet_point_sources(temp_path, '6c', pars, 'tot', tot_df)

### 2.9. Sheet 7: Concentrations

In [22]:
# Read data
in_csv = r'../../../Results/Loads_CSVs/concs_and_flows_rid_20_%s.csv' % year
conc_df = pd.read_csv(in_csv, index_col = 0, encoding='utf-8')

# Convert units
conc_df['Hg_ng/l'] = conc_df['Hg_ng/l'] / 1000 # ng to ug
conc_df[u'NH4-N_µg/l N'] = conc_df[u'NH4-N_µg/l N'] / 1000 # ug to mg
conc_df[u'NO3-N_µg/l N'] = conc_df[u'NO3-N_µg/l N'] / 1000 # ug to mg
conc_df[u'TOTN_µg/l N'] = conc_df[u'TOTN_µg/l N'] / 1000 # ug to mg
conc_df[u'TOTP_µg/l P'] = conc_df[u'TOTP_µg/l P'] / 1000 # ug to mg
conc_df[u'PO4-P_µg/l P'] = conc_df[u'PO4-P_µg/l P'] / 1000 # ug to mg
conc_df['TOC_mg C/l'] = conc_df['TOC_mg C/l'] * 1000 # mg to ug

# Get flags
cols = [i for i in conc_df.columns if i.split('_')[1]=='flag']
lod_df = conc_df[cols]
lod_df.columns = [i.split('_')[0] for i in lod_df.columns]

# Get vals
cols = [i for i in conc_df.columns if ((i.split('_')[0] in lod_df.columns) 
                                       and (i.split('_')[1] != 'flag'))]
conc_df = conc_df[cols]
conc_df.columns = [i.split('_')[0] for i in conc_df.columns]

# Rename
col_dict = {'SPM':'S.P.M.',
            'TOTN':'n',
            'NH4-N':'nh4',
            'NO3-N':'no3',
            'TOTP':'p',
            'PO4-P':'po4'}
for col in col_dict.keys():
    new_col = col_dict[col]
    lod_df[new_col] = lod_df[col]
    conc_df[new_col] = conc_df[col]
    del conc_df[col], lod_df[col]

# Map Excel headings to df cols
par_dict = {'SPM':'S.P.M.',
            'TOC':'TOC',
            'PO4-P':'po4',
            'P-Total':'p',
            'NO3-N':'no3', 
            'NH4-N':'nh4',
            'N-Total':'n',
            'As':'As',
            'Pb':'Pb',
            'Cd':'Cd', 
            'Cu':'Cu',
            'Zn':'Zn',
            'Ni':'Ni',
            'Total Cr':'Cr',
            'Hg':'Hg'}
    
# Map names to stns
names_dict = {'Orkla':29778,
              'Vefsna':29782,
              'Alta':29779,
              'Glomma':29617,
              'Drammenselva':29612,
              u'Numedalslågen':29615,
              'Skienselva':29613,
              'Otra':29614,
              'Orreelva':29783,
              'Vosso':29821}

# Open new file and get sheet
wb = load_workbook(filename=temp_path)
ws = wb['7']

# Get row numbers
row_dict = {}
for item in ws['B12':'B%s' % ws.max_row]:
    # Get cell properties
    cell = item[0]
    name = cell.value
    row = cell.row
    row_dict[name] = row

# Get col numbers
col_dict = {}
for cell in ws['E9':'AK9'][0]:
    # Get cell properties
    par = cell.value
    col = cell.column
    col_dict[par] = col

pars = ['Ni', 'Pb', 'NH4-N', 'Total Cr', 'NO3-N', 
        'Zn', 'As', 'Cd', 'P-Total', 'SPM', 'PO4-P', 
        'N-Total', 'Hg', 'Cu', 'TOC']

# Update spreadsheet
for reg in names_dict.keys():
    for par in pars:
        # Get values from df
        # 1a. Lower average
        vals = conc_df.ix[names_dict[reg], par_dict[par]].values.copy()
        lods = lod_df.ix[names_dict[reg], par_dict[par]].fillna('0').values.copy()
        vals[(lods=='<')] = 0
        val = np.nanmean(vals)
        
        row = row_dict[reg]
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val
        
        # 1b. Upper average
        val = conc_df.ix[names_dict[reg], par_dict[par]].mean()
        row = row_dict[reg]
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 2. Min
        val = conc_df.ix[names_dict[reg], par_dict[par]].min()
        row = row_dict[reg] + 2
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 3. Max
        val = conc_df.ix[names_dict[reg], par_dict[par]].max()
        row = row_dict[reg] + 3
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val   
        
        # 4. N
        n_samp = len(conc_df.ix[names_dict[reg], par_dict[par]].dropna())
        row = row_dict[reg] + 5
        col = col_dict[par]
        ws.cell(row=row, column=col).value = n_samp 

        # 5. N LOD
        n_lod = (~pd.isnull(lod_df.ix[names_dict[reg], par_dict[par]])).sum()
        pct_lod = 100 * float(n_lod)/float(n_samp)
        
        if pct_lod < 30:
            val = 'Yes'
        else:
            val = 'No'
            
        row = row_dict[reg] + 4
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val

        # 6. Std
        val = conc_df.ix[names_dict[reg], par_dict[par]].std()
        row = row_dict[reg] + 7
        col = col_dict[par]
        ws.cell(row=row, column=col).value = val
        
# Save
wb.save(temp_path)

### 2.10. Sheet 9: Discharge

The notebook [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/recalculate_ospar_flows.ipynb) handles the OSPAR flow data. Run this, and then copy the results over to Sheet 9. 